In [9]:
import pandas as pd
import re
import requests
from typing import Optional


In [2]:
rename_map = {
    'site_id': 'site_cd', 'datatype_id': 'parameter_cd', 'site_metadata.site_name': 'site_nm', 
    'datatype_metadata.datatype_common_name': 'parameter_nm', 
    'datatype_metadata.physical_quantity_name': 'alt_parameter_nm', 
    'datatype_metadata.unit_name': 'units', 'site_metadata.lat': 'latitude_dd',
    'site_metadata.longi': 'longitude_dd', 'site_metadata.elevation': 'elevation_m',

}

In [3]:
names = [
    "Gramby", "Green Mountain", "Ruedi", "Williams Fork", "Willow Creek", 
    "Windy Gap", "Wolford", "Flaming Gorge", "Granby", "Green Mountain", 
    "Ruedi", "Williams Fork", "Willow Creek", "Windy Gap", "Wolford Mountain", 
    "Flaming Gorge", "Starvation", "Catamount", "Stagecoach",
    "Blue Mesa", "Crystal", "Morrow Point", "Ridgeway",
    "Powell" 
    ]

parameters = [17, 29, 42, 49] 
pattern = "|".join(re.escape(name) for name in names)

print("Pattern:", pattern)

Pattern: Gramby|Green\ Mountain|Ruedi|Williams\ Fork|Willow\ Creek|Windy\ Gap|Wolford|Flaming\ Gorge|Granby|Green\ Mountain|Ruedi|Williams\ Fork|Willow\ Creek|Windy\ Gap|Wolford\ Mountain|Flaming\ Gorge|Starvation|Catamount|Stagecoach|Blue\ Mesa|Crystal|Morrow\ Point|Ridgeway|Powell


In [8]:
df = pd.read_csv("https://www.usbr.gov/uc/water/hydrodata/reservoir_data/meta.csv")

matches = df[(df["site_metadata.site_name"].str.contains(pattern, case = False, na=False)) &
             (df["site_metadata.db_site_code"] == "UC") &
             (df["datatype_id"].isin(parameters))].sort_values("site_metadata.site_name")

selected = matches[list(rename_map)].rename(columns=rename_map).reset_index(drop=True)
sites = selected.drop_duplicates(subset=['site_cd']).reset_index(drop=True)

print(df.columns)


Index(['site_datatype_id', 'site_id', 'datatype_id', 'site_metadata.site_id',
       'site_metadata.site_name', 'site_metadata.site_common_name',
       'site_metadata.description', 'site_metadata.elevation',
       'site_metadata.lat', 'site_metadata.longi',
       'site_metadata.db_site_code', 'site_metadata.objecttype_id',
       'site_metadata.objecttype_name', 'site_metadata.basin_id',
       'site_metadata.hydrologic_unit', 'site_metadata.river_mile',
       'site_metadata.segment_no', 'site_metadata.state_id',
       'site_metadata.state_code', 'site_metadata.usgs_id',
       'site_metadata.nws_code', 'site_metadata.shef_code',
       'site_metadata.scs_id', 'site_metadata.parent_objecttype_id',
       'site_metadata.parent_site_id', 'datatype_metadata.datatype_id',
       'datatype_metadata.datatype_name',
       'datatype_metadata.datatype_common_name',
       'datatype_metadata.physical_quantity_name', 'datatype_metadata.unit_id',
       'datatype_metadata.unit_name', 'dataty

In [17]:
def fetch_rise_timeseries(
        site_cd: str,
        parameter_cd: str,
        start_date: str,
        end_date: str,
        observed_modeled: str = "observed",
        base_url: str = "https://data.usbr.gov/rise/api/timeseries",
        format: str = "json"
    ) -> Optional[pd.DataFrame]:
    """Fetches time series data from the RISE API for a given site and parameter."""
    
    params = {
        "locationId": site_cd,
        "parameterId": parameter_cd,
        "startDate": start_date,
        "endDate": end_date,
        "observedModeled": observed_modeled,
        "format": format
    }

    headers = {
        "Accept": "application/vnd.api+json"
    }
    try:
        response = requests.get(base_url, params=params, headers=headers)
        response.raise_for_status()  # Raise an error for bad responses

        if format == "json":
            json_data = response.json()
            records = json_data.get("timeSeries", [])
            if not records:
                print(f"No data found for site {site_cd} and parameter {parameter_cd}.")
                return None
            
            df = pd.DataFrame(records)
            return df
        
        elif format == "csv":
            from io import StringIO
            return pd.read_csv(StringIO(response.text))
        
        else:
            raise ValueError("Unsupported format. Use 'json' or 'csv'.")
        
    except requests.RequestException as e:
        print(f"Error fetching data for site {site_cd} and parameter {parameter_cd}: {e}")
        return None

In [19]:
df = fetch_rise_timeseries(
    site_cd='2002',     # Example: Ruedi Reservoir
    parameter_cd='29',      # Example: Storage
    start_date="2024-10-01",
    end_date="2024-12-31"
)

if df is not None:
    print(df.head())

Error fetching data for site 2002 and parameter 29: 404 Client Error: Not Found for url: https://data.usbr.gov/rise/api/timeseries?locationId=2002&parameterId=29&startDate=2024-10-01&endDate=2024-12-31&observedModeled=observed&format=json


In [ ]:
https://www.cbrfc.noaa.gov/wsup/graph/espgraph_hc.html?year=2025&id=CAMC2#
https://www.cbrfc.noaa.gov/wsup/graph/espgraph_hc.html?year=2025&id=CAMC2#